In [9]:
import pandas as pd
import numpy as np
import ast
import re

In [10]:
train_1 = pd.read_csv('test_1.csv', low_memory=False)

In [11]:
list_str_obj_cols = train_1.columns[train_1.dtypes == "object"].tolist()
for str_obj_col in list_str_obj_cols:
    train_1[str_obj_col] = train_1[str_obj_col].astype("category")

list_str_obj_cols = ['loan_id', 'id']
for str_obj_col in list_str_obj_cols:
    train_1[str_obj_col] = train_1[str_obj_col].astype("object")

train_1.dtypes
train_1.to_pickle('test_1_proc.bin')

In [12]:
train_2_1 = pd.read_csv('test_2_1.csv', low_memory=False)

In [13]:
list_str_obj_cols = ['add_431']
for str_obj_col in list_str_obj_cols:
    if str_obj_col in train_2_1.columns:
        train_2_1[str_obj_col] = pd.to_datetime(train_2_1[str_obj_col], errors='coerce')
        
        # Add year, month, day, weekday, and weekend columns
        train_2_1[f'{str_obj_col}_year'] = train_2_1[str_obj_col].dt.year
        train_2_1[f'{str_obj_col}_month'] = train_2_1[str_obj_col].dt.month
        train_2_1[f'{str_obj_col}_day'] = train_2_1[str_obj_col].dt.day
        train_2_1[f'{str_obj_col}_weekday'] = train_2_1[str_obj_col].dt.weekday
        train_2_1[f'{str_obj_col}_is_weekend'] = train_2_1[str_obj_col].dt.weekday >= 5

        # Drop the original date column
        train_2_1.drop(columns=[str_obj_col], inplace=True)

list_str_obj_cols = ['add_671', 'add_672', 'add_673', 'add_675', 'add_676', 'add_677']

def safe_eval_array(x):
    try:
        x = re.sub(r'\b0+(\d)', r'\1', x)
        x = ast.literal_eval(x)
        return ast.literal_eval(x)
    except:
        return []

def unwrap_arrays(data, columns):
    for col in columns:
        # Step 1: Convert string to lists using safe_eval_array
        mySeries = data[col].values
        mySeries = mySeries.tolist()
        mySeries = [safe_eval_array(x) for x in mySeries]
        mySeries = pd.Series(mySeries)
        # print(col)
        # print(mySeries)

        # Step 3: Find the maximum length of the arrays in the column
        max_len = mySeries.apply(len).max()

        # Step 4: Create separate columns for each element in the arrays
        exploded_cols = pd.DataFrame(
            mySeries.apply(lambda x: x + [np.nan] * (max_len - len(x))).tolist(),
            index=data.index,
            columns=[f'{col}_elem_{i+1}' for i in range(max_len)]
        )

        # Step 5: Calculate the mean and length of the arrays
        data[f'{col}_mean'] = mySeries.apply(lambda x: np.mean(x) if len(x) > 0 else np.nan)
        data[f'{col}_len'] = mySeries.apply(len)

        # Step 6: Concatenate the exploded columns to the original dataframe
        data = pd.concat([data, exploded_cols], axis=1)

        # Step 7: Drop the original column
        data.drop(columns=[col], inplace=True)

    return data

train_2_1 = unwrap_arrays(train_2_1, list_str_obj_cols)

train_2_1.to_pickle('test_2_1_proc.bin')

In [14]:
train_2_1 = pd.read_csv('test_2_2.csv', low_memory=False)

In [15]:
list_str_obj_cols = ['add_431']
for str_obj_col in list_str_obj_cols:
    if str_obj_col in train_2_1.columns:
        train_2_1[str_obj_col] = pd.to_datetime(train_2_1[str_obj_col], errors='coerce')
        
        # Add year, month, day, weekday, and weekend columns
        train_2_1[f'{str_obj_col}_year'] = train_2_1[str_obj_col].dt.year
        train_2_1[f'{str_obj_col}_month'] = train_2_1[str_obj_col].dt.month
        train_2_1[f'{str_obj_col}_day'] = train_2_1[str_obj_col].dt.day
        train_2_1[f'{str_obj_col}_weekday'] = train_2_1[str_obj_col].dt.weekday
        train_2_1[f'{str_obj_col}_is_weekend'] = train_2_1[str_obj_col].dt.weekday >= 5

        # Drop the original date column
        train_2_1.drop(columns=[str_obj_col], inplace=True)

list_str_obj_cols = ['add_671', 'add_672', 'add_673', 'add_675', 'add_676', 'add_677']

def safe_eval_array(x):
    try:
        x = re.sub(r'\b0+(\d)', r'\1', x)
        x = ast.literal_eval(x)
        return ast.literal_eval(x)
    except:
        return []

def unwrap_arrays(data, columns):
    for col in columns:
        # Step 1: Convert string to lists using safe_eval_array
        mySeries = data[col].values
        mySeries = mySeries.tolist()
        mySeries = [safe_eval_array(x) for x in mySeries]
        mySeries = pd.Series(mySeries)
        # print(col)
        # print(mySeries)

        # Step 3: Find the maximum length of the arrays in the column
        max_len = mySeries.apply(len).max()

        # Step 4: Create separate columns for each element in the arrays
        exploded_cols = pd.DataFrame(
            mySeries.apply(lambda x: x + [np.nan] * (max_len - len(x))).tolist(),
            index=data.index,
            columns=[f'{col}_elem_{i+1}' for i in range(max_len)]
        )

        # Step 5: Calculate the mean and length of the arrays
        data[f'{col}_mean'] = mySeries.apply(lambda x: np.mean(x) if len(x) > 0 else np.nan)
        data[f'{col}_len'] = mySeries.apply(len)

        # Step 6: Concatenate the exploded columns to the original dataframe
        data = pd.concat([data, exploded_cols], axis=1)

        # Step 7: Drop the original column
        data.drop(columns=[col], inplace=True)

    return data

train_2_1 = unwrap_arrays(train_2_1, list_str_obj_cols)

train_2_1.to_pickle('test_2_2_proc.bin')

In [16]:
train_1 = pd.read_pickle('test_1_proc.bin')
train_2_1 = pd.read_pickle('test_2_1_proc.bin')
train_2_2 = pd.read_pickle('test_2_2_proc.bin')

In [21]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer  # Needed to enable IterativeImputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
import pickle

def impute_numeric_categorical(data, imputers):# -> Any:
    # Step 1: Separate numerical, categorical, and object columns
    num_cols = data.select_dtypes(include=['number']).columns
    cat_cols = data.select_dtypes(include=['category']).columns

    imputer_num = imputers['numerical']
    encoder = imputers['encoder']
    imputer_cat = imputers['categorical']

    # Step 2: Impute numerical columns with IterativeImputer
    if len(num_cols) > 0:
        num_data = data[num_cols]
        imputed_num_data = imputer_num.transform(num_data[imputer_num.feature_names_in_])
        data[imputer_num.feature_names_in_] = imputed_num_data

    # Step 3: Impute categorical columns with SimpleImputer (most frequent strategy)
    if len(cat_cols) > 0:
        # Encode categorical columns temporarily using OrdinalEncoder
        encoded_cat_data = encoder.transform(data[encoder.feature_names_in_])

        # Impute encoded categorical columns with SimpleImputer
        imputed_cat_data = imputer_cat.transform(encoded_cat_data)

        # Convert back to original categories using the encoder
        imputed_cat_data = encoder.inverse_transform(imputed_cat_data)
        data[encoder.feature_names_in_] = imputed_cat_data

    return data


with open("imputers.pkl", "rb") as f:
    imputers = pickle.load(f)

imputed_data = impute_numeric_categorical(train_1, imputers['train_1'])

imputed_data.to_pickle('test_1_imputed.bin')

imputed_data = impute_numeric_categorical(train_2_1, imputers['train_2_1'])

imputed_data.to_pickle('test_2_1_imputed.bin')

imputed_data = impute_numeric_categorical(train_2_2, imputers['train_2_2'])

imputed_data.to_pickle('test_2_2_imputed.bin')

In [2]:
import pandas as pd

In [3]:
# Load the datasets
train_1 = pd.read_pickle('test_1_imputed.bin')
train_2_1 = pd.read_pickle('test_2_1_imputed.bin')
train_2_2 = pd.read_pickle('test_2_2_imputed.bin')

In [4]:

# Step 1: Aggregate train_2_1 at 'id' level
adict = {col: ['mean', 'sum', 'max', 'min'] for col in train_2_1.columns if col not in ['id', 'add_431']}
agg_train_2_1 = train_2_1.groupby('id').agg(
    adict
).reset_index()

# Flatten column names after aggregation
agg_train_2_1.columns = ['_'.join(col).strip('_') for col in agg_train_2_1.columns]

adict = {col: ['mean', 'sum', 'max', 'min'] for col in train_2_2.columns if col not in ['id', 'add_431']}
agg_train_2_2 = train_2_2.groupby('id').agg(
    adict
).reset_index()

# Flatten column names after aggregation
agg_train_2_2.columns = ['t2_'+'_'.join(col).strip('_') for col in agg_train_2_2.columns]
agg_train_2_2.rename(columns={'t2_id': 'id'}, inplace=True)

In [5]:
train_1 = train_1.merge(agg_train_2_1, on='id', how='left')
train_1 = train_1.merge(agg_train_2_2, on='id', how='left')

In [6]:
del agg_train_2_1
del agg_train_2_2

In [9]:
import pandas as pd
import numpy as np
from sklearn import impute
from sklearn.experimental import enable_iterative_imputer  # Needed to enable IterativeImputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
import pickle

def impute_numeric_categorical(data, imputers2):# -> Any:
    # Step 1: Separate numerical, categorical, and object columns
    num_cols = data.select_dtypes(include=['number']).columns
    cat_cols = data.select_dtypes(include=['category']).columns

    imputer_num = imputers2['numerical']
    encoder = imputers2['encoder']
    imputer_cat = imputers2['categorical']

    # Step 2: Impute numerical columns with IterativeImputer
    if len(num_cols) > 0:
        num_data = data[num_cols]
        imputed_num_data = imputer_num.transform(num_data[imputer_num.feature_names_in_])
        data[imputer_num.feature_names_in_] = imputed_num_data

    # Step 3: Impute categorical columns with SimpleImputer (most frequent strategy)
    # if len(cat_cols) > 0:
    #     # Encode categorical columns temporarily using OrdinalEncoder
    #     encoded_cat_data = encoder.transform(data[encoder.feature_names_in_])

    #     # Impute encoded categorical columns with SimpleImputer
    #     imputed_cat_data = imputer_cat.transform(encoded_cat_data)

    #     # Convert back to original categories using the encoder
    #     imputed_cat_data = encoder.inverse_transform(imputed_cat_data)
    #     data[encoder.feature_names_in_] = imputed_cat_data

    return data


with open('imputers2.pkl', 'rb') as f:
    imputers2 = pickle.load(f)

# Apply imputation to numerical and categorical columns only
imputed_data = impute_numeric_categorical(train_1, imputers2)

imputed_data.to_pickle('test_1_agg_imputed.bin')


In [2]:
import pandas as pd
import pickle

In [8]:
imputed_data = pd.read_pickle('test_1_agg_imputed.bin')

In [ ]:
imputed_data

In [9]:
imputed_data = pd.merge(imputed_data, oot, left_on="loan_id" ,right_on="loan_id")

In [12]:
imputed_data = imputed_data.drop(columns=['Tag'])

In [ ]:
imputed_data

In [14]:
X  = imputed_data.drop(columns=['id', 'loan_id'])
labels = imputed_data['loan_id']

In [ ]:
labels

In [3]:
oot = pd.read_csv("test_oot_map.csv")

In [ ]:
oot

In [5]:
oot = oot[oot['Tag'] == 'private']

In [ ]:
oot

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def preprocess_data(df, scaler, ohe):
    # Step 1: Detect categorical and numerical columns
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns

    # Step 2: One-Hot Encode categorical columns
    if len(cat_cols) > 0:
        encoded_cat_data = ohe.transform(df[ohe.feature_names_in_])
        encoded_cat_df = pd.DataFrame(encoded_cat_data, columns=ohe.get_feature_names_out(ohe.feature_names_in_), index=df.index)
    else:
        encoded_cat_df = pd.DataFrame(index=df.index)  # Empty DataFrame if no categorical columns

    # Step 3: Standard Scale numerical columns
    if len(num_cols) > 0:
        scaled_num_data = scaler.transform(df[scaler.feature_names_in_])
        scaled_num_df = pd.DataFrame(scaled_num_data, columns=scaler.feature_names_in_, index=df.index)
    else:
        scaled_num_df = pd.DataFrame(index=df.index)  # Empty DataFrame if no numerical columns

    # Step 4: Concatenate the processed categorical and numerical data
    X = pd.concat([scaled_num_df, encoded_cat_df], axis=1)

    return X

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('ohe.pkl', 'rb') as f:
    ohe = pickle.load(f)

# Preprocess the data and split into X and y
X = preprocess_data(X, scaler, ohe)

In [16]:
with open("X_test.pkl", "wb") as f:
    pickle.dump(X, f)

with open("labels.pkl", "wb") as f:
    pickle.dump(labels, f)

In [ ]:
with open("xgb_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("lgbm_model.pkl", "rb") as f:
    model2 = pickle.load(f)

with open("catboost_model.pkl", "rb") as f:
    model3 = pickle.load(f)

with open("final_calibrated_model.pkl", "rb") as f:
    metamodel = pickle.load(f)

with open("X_test.pkl", "rb") as f:
    X = pickle.load(f)

from sklearn.metrics import roc_auc_score

# Step 1: Predict the probabilities
y_pred = model.predict(X)
y_pred2 = model2.predict(X)
y_pred3 = model3.predict(X)

pred_df = pd.DataFrame({
    'XGB_Preds': y_pred,
    'LGBM_Preds': y_pred2,
    'CatBoost_Preds': y_pred3
})

# Step 2: Meta-model prediction
final_preds = metamodel.predict_proba(pred_df)[:, 1]

In [ ]:
pred_df